In [43]:
#!pip install autogluon
#!pip install pandas

In [44]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random

In [45]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [48]:
x_train_c = pd.read_csv('cleaned_data_quarterly/C/x_train_c.csv')
y_train_c = pd.read_csv('cleaned_data_quarterly/C/train_c.csv')
x_test_c = pd.read_csv('cleaned_data_quarterly/C/x_test_c.csv')

In [53]:
x_train_c_combined = x_train_c.merge(y_train_c, left_on='date_forecast', right_on='time', how='left')
x_train_c_combined['observed'] = x_train_c_combined['calc_year'].isna().astype(int)
train_data_c = x_train_c_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)
x_test_c['observed'] = x_test_c['calc_year'].isna().astype(int)
test_data_c = x_test_c.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)

# Trying with validation set equal to half of the estimated data


In [56]:
def split_dataset(train_data, observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set contains the last half of the rows where observed = 0,
    and the training set contains the rest.

    :param train_data: The original training dataset as a pandas DataFrame.
    :param observed_column: The name of the column that indicates if the row is observed.
    :return: A tuple (training_set, validation_set)
    """

    # Filter rows where observed = 0
    observed_zero = train_data[train_data[observed_column] == 0]

    # Split the filtered dataset into two
    training_set_estimated, validation_set = train_test_split(observed_zero,test_size=0.5, random_state=42)
    

    # Combine the first half of observed_zero with the rest of the data where observed != 0
    training_set = pd.concat([train_data[train_data[observed_column] != 0], training_set_estimated])

    return training_set, validation_set


train_data_c, val_data_c = split_dataset(train_data_c, 'observed')


In [ ]:
test_data_c['snow_density:kgm3_Q0'] = np.nan
test_data_c['snow_density:kgm3_Q1'] = np.nan
test_data_c['snow_density:kgm3_Q2'] = np.nan
test_data_c['snow_density:kgm3_Q3'] = np.nan
train_data_c.drop(columns= ['index'], inplace= True)

In [61]:

#val_data_a.head()

In [64]:
predictor_c= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_c,tuning_data = val_data_c, verbosity = 2,presets='best_quality',excluded_model_types=['CatBoost'], time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231106_133907\"
Presets specified: ['best_quality']
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231106_133907\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   682.92 GB / 1022.87 GB (66.8%)
Train Data Rows:    21660
Train Data Columns: 189
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\uti

In [67]:
predictor_c.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-18.942188,113.369493,1339.310277,0.000000,0.449101,3,True,20
1,ExtraTreesMSE_BAG_L2,-19.148157,109.214885,989.997355,1.852356,22.611369,2,True,15
2,WeightedEnsemble_L2,-19.233810,100.947357,452.670734,0.001000,0.438659,2,True,10
3,LightGBMXT_BAG_L1,-19.290144,48.633917,176.933922,48.633917,176.933922,1,True,3
4,RandomForestMSE_BAG_L2,-19.308938,109.202880,1099.407938,1.840351,132.021953,2,True,13
5,XGBoost_BAG_L2,-19.444459,107.684995,984.918767,0.322466,17.532782,2,True,17
6,NeuralNetFastAI_BAG_L2,-19.444874,108.052308,1025.398986,0.689779,58.013000,2,True,16
7,LightGBMLarge_BAG_L2,-19.594658,107.739722,1012.846740,0.377192,45.460754,2,True,19
8,LightGBM_BAG_L2,-19.701149,107.514121,978.619665,0.151592,11.233679,2,True,12
9,CatBoost_BAG_L2,-19.897730,107.470618,1025.390336,0.108089,58.004350,2,True,14


In [74]:
y_pred_c = predictor_c.predict(test_data_c)
y_pred_c=pd.DataFrame(y_pred_c,columns=['pv_measurement'])
y_pred_c #print the DataFrame

c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a f

,pv_measurement
0,-0.182750
1,-0.163155
2,-0.155448
3,1.503704
4,12.314628
...,...
715,69.798340
716,44.278572
717,17.382914
718,2.631103


In [76]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_c.to_csv(os.path.join(output_dir, 'y_pred_c.csv'))

,prediction
0,1.016324
1,0.851993
2,1.456194
3,61.065281
4,306.626099
...,...
2155,69.798340
2156,44.278572
2157,17.382914
2158,2.631103
